# Multi-layer LSTM for Music Generation (with Magenta)

In [1]:
import os
from collections import namedtuple
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Bidirectional, BatchNormalization, Dropout, Activation
from magenta.scripts.convert_dir_to_note_sequences import convert_directory
from note_seq import music_pb2

Note = namedtuple("Note", ["flag", "pitch", "instrument"])

test_file = 'test_sequences.tfrecord'
train_file = 'train_sequences.tfrecord'
valid_file = 'valid_sequences.tfrecord'

In [3]:
def create_network(network_input, n_vocab):
    """ Define constants """
    hidden_layers = 256
    dropout = 0.4
    temp = 0.6
    
    """ Initializing model """
    model = Sequential()
    
    """ Adding LSTM Layers to Model """
    model.add(
        Bidirectional(
            LSTM(
                hidden_layers,
                dropout=dropout,
                return_sequences=True,
                input_shape=(network_input.shape[1], network_input.shape[2])
            )
        )
    )
    model.add(
        Bidirectional(
            LSTM(
                hidden_layers,
                dropout=dropout,
                return_sequences=True
            )
        )
    )
    model.add(
        Bidirectional(
            LSTM(
                hidden_layers,
                dropout=dropout
            )
        )
    )
    
    """ Add other layers after LSTM """
    model.add(BatchNormalization())
    model.add(Dropout(dropout))
    model.add(Dense(hidden_layers // 2))
    model.add(Activation('relu'))
    model.add(BatchNormalization())
    model.add(Dropout(dropout))
    model.add(Dense(n_vocab))
    model.add(Lambda(lambda x: x / temp))
    model.add(Softmax())
    
    """ Define the optimizer and loss function for the model """
    model.compile(optimizer='adam', loss='categorical_crossentropy')
    
    return model
    

In [7]:
convert_directory(os.path.join(os.getcwd(), 'nesmdb_midi', 'test'), test_file, True)
convert_directory(os.path.join(os.getcwd(), 'nesmdb_midi', 'train'), train_file, True)
convert_directory(os.path.join(os.getcwd(), 'nesmdb_midi', 'valid'), valid_file, True)

In [21]:
reader = tf.data.TFRecordDataset(train_file)
sequences = []
for sequence in reader:
    sequences.append(sequence)
data = music_pb2.NoteSequence.FromString(sequences[0].numpy())
data.notes.sort(key=lambda e: e.start_time)
# music_pb2.NoteSequence.FromString(sequences[1].numpy()).notes[0].instrument
data.instrument_infos[3].name
data

id: "/id/midi/train/aeaf377502e754ed80bc5cebf6c88b5b6f3b2094"
filename: "135_GanbareGoemonGaiden2_TenkanoZaih__63_64AntarcticResearchVesselTravel.mid"
collection_name: "train"
ticks_per_quarter: 22050
time_signatures {
  numerator: 4
  denominator: 4
}
time_signatures {
  time: 6.3920408163265305
  numerator: 1
  denominator: 1
}
tempos {
  qpm: 120.0
}
notes {
  pitch: 72
  velocity: 2
  start_time: 0.0011564625850340137
  end_time: 0.2669614512471655
  program: 80
}
notes {
  pitch: 72
  velocity: 7
  start_time: 0.0022675736961451248
  end_time: 0.13408163265306122
  instrument: 1
  program: 81
}
notes {
  pitch: 56
  velocity: 1
  start_time: 0.005215419501133787
  end_time: 0.13541950113378684
  instrument: 2
  program: 38
}
notes {
  pitch: 75
  velocity: 7
  start_time: 0.13408163265306122
  end_time: 0.2676643990929705
  instrument: 1
  program: 81
}
notes {
  pitch: 75
  velocity: 2
  start_time: 0.2669614512471655
  end_time: 0.4000907029478458
  program: 80
}
notes {
  pitch